**RAG with LangChain**

In [1]:
!pip install langchain langchain_openai langchain_experimental pandas numpy tabulate openpyxl -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 103.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.0 MB/s eta 0:00:00


In [2]:
from langchain_openai import ChatOpenAI
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.agents.agent_types import AgentType
from langchain.schema.output_parser import StrOutputParser
from langchain.output_parsers import PandasDataFrameOutputParser
from langchain.prompts import ChatPromptTemplate, PromptTemplate
import pandas as pd
import numpy as np
import os

In [3]:
df= pd.read_excel("titanic.xlsx")


In [4]:
df.head()

,Id_Pasajero,Survived,Clase,Nombre,Sexo,Edad,Familiares,Tiquete,Tarifa,Cabina,Embarcado
0,1,0,3,"Braund, Mr. Owen Harris",Masculino,22.0,0,A/5 21171,7.25,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",Femenino,38.0,0,PC 17599,71.28,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",Femenino,26.0,0,STON/O2. 3101282,7.93,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",Femenino,35.0,0,113803,53.10,C123,S
4,5,0,3,"Allen, Mr. William Henry",Masculino,35.0,0,373450,8.05,NaN,S


In [5]:
os.environ ["OPENAI_API_KEY"] = "####################################################################"
model = "gpt-3.5-turbo-16k"


In [6]:
llm_model = ChatOpenAI(model=model, temperature=0, verbose=True, streaming=True)



In [ ]:
df_parser = PandasDataFrameOutputParser(dataframe=df)


In [ ]:
print(df_parser.get_format_instructions())

The output should be formatted as a string as the operation, followed by a colon, followed by the column or row to be queried on, followed by optional array parameters.
1. The column names are limited to the possible columns below.
2. Arrays must either be a comma-separated list of numbers formatted as [1,3,5], or it must be in range of numbers formatted as [0..4].
3. Remember that arrays are optional and not necessarily required.
4. If the column is not in the possible columns or the operation is not a valid Pandas DataFrame operation, return why it is invalid as a sentence starting with either "Invalid column" or "Invalid operation".

As an example, for the formats:
1. String "column:num_legs" is a well-formatted instance which gets the column num_legs, where num_legs is a possible column.
2. String "row:1" is a well-formatted instance which gets row 1.
3. String "column:num_legs[1,2]" is a well-formatted instance which gets the column num_legs for rows 1 and 2, where num_legs is a p

La salida debe estar formateada como una cadena con la operación, seguida de dos puntos, seguida de la colurnna o fila a consultar, seguida de parámetros opcionales de arreglo.
1. Los nombres de las columnas se limitan a las posibles columnas a continuación.
2. Los arreglos deben ser una lista de números separados por comas formateados como [1,3,5], o deben estar en un rango de números formateados como [0..4].
3. Recuerda que los arreglos son opcionales y no necesariamente requeridas.
4. Si la columna no está en las columnas posibles o la operación no es una operación válida de Pandas DataFrame, devuelve la razón de por qué es inválida como una oración que comience con "Columna inválida" o "Operación inválida".

Como ejemplo, para los formatos:
1. La cadena "column:num_legs" es un ejemplo bien formateado que obtiene la columna num legs, donde num_legs es una columna posible.
2. La cadena "row:1" es un ejemplo bien formateado que obtiene la fila 1.
3. La cadena "column:num_legs[1,2]" es un ejemplo bien formateado que obtiene la columna num legs para las filas 1 y 2, donde num_legs es una columna posible.
4. La cadena "row:1[num_legs]" es un ejemplo bien formateado que obtiene la fila 1, pero solo para la columna num legs, donde num_legs es una columna
posible.
5. La cadena "mean:num_legs[1..3] es un ejemplo bien formateado que toma la media de num legs desde las filas 1 a 3, donde num legs es una columna
posible y mean es una operación válida de Pandas DataFrame.


In [ ]:
prompt = PromptTemplate(
    template = "Responde al usuario su consulta.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": df_parser.get_format_instructions()},
)

In [ ]:
prompt = PromptTemplate(
    template = "Responde al usuario su consulta. La respuesta DEBE estar en el formato:\n{format_instructions}\nSolo incluye el comando y los parámetros necesarios. NO incluyas texto adicional.\nConsulta: {query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": df_parser.get_format_instructions()},
)

In [ ]:
chain = prompt | llm_model | df_parser

In [ ]:
chain.invoke({"query": "dame la primera fila."})

{'0': Id_Pasajero                          1
 Survived                             0
 Clase                                3
 Nombre         Braund, Mr. Owen Harris
 Sexo                         Masculino
 Edad                              22.0
 Familiares                           0
 Tiquete                      A/5 21171
 Tarifa                            7.25
 Cabina                             NaN
 Embarcado                            S
 Name: 0, dtype: object}

In [ ]:
chain.invoke({"query": "dame la media de la columna edad."})

{'mean': np.float64(29.69911764705882)}

In [ ]:
df["Edad"].mean()

np.float64(29.69911764705882)

In [ ]:
pandas_df_agent = create_pandas_dataframe_agent(llm=llm_model, df=df, verbose=True,
                                                handle_parsing_errors=True,
                                                agent_type=AgentType.OPENAI_FUNCTIONS,
                                                return_intermediate_steps=True,
                                                allow_dangerous_code=True)

/usr/local/lib/python3.11/dist-packages/langchain_experimental/agents/agent_toolkits/pandas/base.py:283: UserWarning: Received additional kwargs {'handle_parsing_errors': True} which are no longer supported.
  warnings.warn(


In [ ]:
pandas_result = pandas_df_agent({"input": "que tamaño tiene de df?"})

<ipython-input-26-1269879456>:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  pandas_result = pandas_df_agent({"input": "que tamaño tiene de df?"})




> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'df.shape'}`


(891, 11)El dataframe `df` tiene 891 filas y 11 columnas.

> Finished chain.


In [ ]:
df.shape

(891, 11)

#**Pasar Excel a SQl y comparar consulta... ?**

In [7]:
import sqlite3

In [8]:
# Conectar o crear la base de datos SQLite
conn = sqlite3.connect('titanic.db')
cursor = conn.cursor()

In [9]:
# Crear la tabla en la base de datos SQLite
cursor.execute("""
CREATE TABLE IF NOT EXISTS passengers (
    Id_Pasajero INTEGER PRIMARY KEY,
    Survived INTEGER,
    Clase INTEGER,
    Nombre TEXT,
    Sexo TEXT,
    Edad INTEGER,
    Familia INTEGER,
    Tiquete TEXT,
    Tarifa REAL,
    Cabina TEXT,
    Embarque TEXT
);
""")

# Commit para guardar los cambios
conn.commit()

In [10]:
# Convertir el DataFrame a una lista de tuplas para insertar en la base de datos
data_tuples = [tuple(x) for x in df.to_numpy()]

# Insertar los datos en la tabla
cursor.executemany("""
INSERT INTO passengers (Id_Pasajero, Survived, Clase, Nombre, Sexo, Edad,
                        Familia, Tiquete, Tarifa, Cabina, Embarque)
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
""", data_tuples)

# Commit para guardar los cambios
conn.commit()

# Cerrar la conexión
conn.close()


In [14]:
pip install --upgrade langchain


In [20]:
import sqlite3
from langchain.llms import OpenAI
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.prompts import PromptTemplate
from langchain.sql_database import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from sqlalchemy import create_engine # Import create_engine

# Conectar a la base de datos SQLite usando SQLAlchemy engine
# The URL format is 'sqlite:///your_database_name.db' for a file-based database
engine = create_engine('sqlite:///titanic.db')

# Crear una instancia de la base de datos SQL a partir del engine de SQLAlchemy
db = SQLDatabase(engine)

# Configurar el modelo de OpenAI
llm = OpenAI(temperature=0)

# Crear el SQLDatabaseChain
database_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True) # Adding verbose=True for debugging


# Realizar una consulta de ejemplo
query = "SELECT * FROM passengers WHERE Survived = 1;"
response = database_chain.run(query)
print(response)

/usr/local/lib/python3.11/dist-packages/langchain_experimental/sql/base.py:77: UserWarning: Directly instantiating an SQLDatabaseChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(
/tmp/ipython-input-20-2871579516.py:25: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = database_chain.run(query)




> Entering new SQLDatabaseChain chain...
SELECT * FROM passengers WHERE Survived = 1;
SQLQuery:SELECT "Id_Pasajero", "Clase", "Nombre", "Sexo", "Edad", "Familia", "Tiquete", "Tarifa", "Cabina", "Embarque" FROM passengers WHERE Survived = 1 LIMIT 5;
SQLResult: [(2, 1, 'Cumings, Mrs. John Bradley (Florence Briggs Thayer)', 'Femenino', 38, 0, 'PC 17599', 71.28, 'C85', 'C'), (3, 3, 'Heikkinen, Miss. Laina', 'Femenino', 26, 0, 'STON/O2. 3101282', 7.93, None, 'S'), (4, 1, 'Futrelle, Mrs. Jacques Heath (Lily May Peel)', 'Femenino', 35, 0, '113803', 53.1, 'C123', 'S'), (9, 3, 'Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)', 'Femenino', 27, 2, '347742', 11.13, None, 'S'), (10, 2, 'Nasser, Mrs. Nicholas (Adele Achem)', 'Femenino', 14, 0, '237736', 30.07, None, 'C')]
Answer:There were 5 female passengers who survived.
> Finished chain.
There were 5 female passengers who survived.


#**Integración con el Agente LangChain**
El siguiente paso será integrar el agente LangChain para que pueda decidir, dependiendo de la consulta, si usar SQL o el archivo XLSX como fuente de datos.

Vamos a implementar un agente LangChain que reciba preguntas en lenguaje natural y decida automáticamente si la consulta debe hacerse en la base de datos SQL o en el archivo XLSX.

**Crear las Herramientas**
Necesitamos dos herramientas para el agente LangChain:

- SQLTool: Para consultar la base de datos SQLite.
- ExcelTool: Para consultar el archivo XLSX.

**Herramienta SQL para LangChain**

Ya tienes la configuración de SQLDatabaseChain para hacer consultas a la base de datos. Ahora definimos una herramienta para LangChain que interactúa con esta base de datos:

In [22]:
# Herramienta SQL
def query_sql_database(query):
    result = database_chain.run(query)
    return result

# Crear la herramienta SQL para LangChain
sql_tool = Tool(
    name="SQL Query",
    func=query_sql_database,
    description="Utiliza la base de datos SQL para responder preguntas relacionadas con los pasajeros del Titanic."
)


**Herramienta Excel para LangChain**

Ahora vamos a definir una herramienta similar para el archivo XLSX que ya tienes:

In [23]:
# Herramienta Excel para LangChain
def query_excel_data(query):
    filtered_df = excel_data.query(query)
    return filtered_df.to_dict(orient='records')

# Crear la herramienta Excel para LangChain
excel_tool = Tool(
    name="Excel Query",
    func=query_excel_data,
    description="Utiliza el archivo Excel Titanic para responder preguntas relacionadas con los pasajeros."
)


**Agente LangChain**

Ahora que tenemos las herramientas para SQL y Excel, vamos a crear un agente que decida qué herramienta utilizar dependiendo de la consulta.

In [24]:
# Herramientas disponibles para el agente
tools = [sql_tool, excel_tool]

# Crear el agente LangChain
agent = initialize_agent(
    tools, llm, agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

# Realizar una consulta ejemplo
query_1 = "¿Cuántos sobrevivientes había en el Titanic?"
query_2 = "¿Cuáles eran las edades de los pasajeros que sobrevivieron?"

# Ejemplo de uso
response_1 = agent.run(query_1)
response_2 = agent.run(query_2)

print(f"Respuesta 1: {response_1}")
print(f"Respuesta 2: {response_2}")



/tmp/ipython-input-24-4118640763.py:5: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(




> Entering new AgentExecutor chain...
 Para responder a esta pregunta, necesitamos encontrar el número de pasajeros que sobrevivieron en el Titanic.
Action: SQL Query
Action Input: SELECT COUNT(*) FROM passengers WHERE survived = 1

> Entering new SQLDatabaseChain chain...
SELECT COUNT(*) FROM passengers WHERE survived = 1
SQLQuery:SELECT COUNT(*) FROM passengers WHERE survived = 1
SQLResult: [(342,)]
Answer:342
> Finished chain.

Observation: 342
Thought: Ahora sabemos que hubo 342 sobrevivientes en el Titanic.
Final Answer: 342 sobrevivientes en el Titanic.

> Finished chain.


> Entering new AgentExecutor chain...
 Para responder a esta pregunta, necesitamos encontrar los pasajeros que sobrevivieron y luego obtener sus edades.
Action: SQL Query
Action Input: SELECT Age FROM passengers WHERE Survived = 1

> Entering new SQLDatabaseChain chain...
SELECT Age FROM passengers WHERE Survived = 1
SQLQuery:SELECT "Edad" FROM passengers WHERE "Survived" = 1 LIMIT 5
SQLResult: [(38,), (26,)

**Funcionamiento del Agente**

- El agente LangChain selecciona automáticamente la herramienta correcta (SQL o Excel) en función de la pregunta.

- Si la pregunta se puede responder mejor con SQL (por ejemplo, "¿Cuántos sobrevivientes había?"), la consulta se enviará a la base de datos SQLite.

- Si la pregunta requiere un análisis más detallado o específico, como filtrar por edad de los pasajeros sobrevivientes, el agente utilizará el archivo XLSX.

**Lo que paso...**

- Consulta sobre los sobrevivientes: El agente detectó que la consulta requería contar los sobrevivientes, así que ejecutó la consulta **SELECT COUNT(*) FROM passengers WHERE survived = 1** en la base de datos SQL y obtuvo la respuesta 342.

- Consulta sobre las edades de los sobrevivientes: Luego, para la pregunta sobre las edades, el agente entendió que necesitaba obtener las edades de los pasajeros sobrevivientes y ejecutó la consulta **SELECT Age FROM passengers WHERE Survived = 1** en la base de datos SQL, devolviendo una lista de edades.



**Visualización de los resultados...**

Podemos también agregar una visualización sencilla de los resultados si es necesario para tener una interfaz más amigable. Por ejemplo, si los resultados son largos o complejos, podemos mostrar una tabla o hacer un resumen de los datos.

In [29]:
import pandas as pd

# Herramienta Excel para LangChain
def query_excel_data(query):
    # Use the globally available df DataFrame
    # Change 'Pclass' to 'Clase' to match the DataFrame column name
    filtered_df = df.query(query)
    return filtered_df.to_dict(orient='records')

# Crear la herramienta Excel para LangChain
excel_tool = Tool(
    name="Excel Query",
    func=query_excel_data,
    description="Utiliza el archivo Excel Titanic para responder preguntas relacionadas con los pasajeros."
)

# Mostrar resultados como tabla para consultas con Excel
def display_results_as_table(results):
    df = pd.DataFrame(results)
    print(df.head())  # Mostrar las primeras filas

# Si los resultados vienen de Excel
# Change 'Pclass' to 'Clase' in the query string
response_excel = query_excel_data("Survived == 1 and Clase == 1")
display_results_as_table(response_excel)

   Id_Pasajero  Survived  Clase  \
0            2         1      1   
1            4         1      1   
2           12         1      1   
3           24         1      1   
4           32         1      1   

                                              Nombre       Sexo  Edad  \
0  Cumings, Mrs. John Bradley (Florence Briggs Th...   Femenino  38.0   
1       Futrelle, Mrs. Jacques Heath (Lily May Peel)   Femenino  35.0   
2                           Bonnell, Miss. Elizabeth   Femenino  58.0   
3                       Sloper, Mr. William Thompson  Masculino  28.0   
4     Spencer, Mrs. William Augustus (Marie Eugenie)   Femenino   NaN   

   Familiares   Tiquete  Tarifa Cabina Embarcado  
0           0  PC 17599   71.28    C85         C  
1           0    113803   53.10   C123         S  
2           0    113783   26.55   C103         S  
3           0    113788   35.50     A6         S  
4           0  PC 17569  146.52    B78         C  
